# Congestion Games and Weighted Congestion Games

In this tutorial, we will explore **congestion games**, a fundamental class of games in algorithmic game theory. Congestion games model scenarios where multiple self-interested players share a common set of resources (such as routes in a network), and the cost incurred by each player depends only on the number of players choosing the same resources.

An important theoretical result is that all **unweighted congestion games** are exact potential games, which guarantees the existence of at least one pure Nash equilibrium. However, this property does not generally hold for **weighted congestion games**, where players have different weights (demands) affecting the resources. In the weighted case, pure Nash equilibria are not guaranteed, but mixed Nash equilibria still exist.



## 1. Defining the Network and Cost Functions

Consider a simple network with a source node $s$, an intermediate node $v$, and a sink node $t$. The setup is as follows:
- Two edges from $s$ to $v$: $e_1$ with cost $c_1(x) = x + 33$, and $e_2$ with cost $c_2(x) = 3x^2$
- Two edges from $v$ to $t$: $e_3$ with cost $c_3(x) = 13x$, and $e_4$ with cost $c_4(x) = x^2 + 44$
- One direct edge from $s$ to $t$: $e_5$ with cost $c_5(x) = 47x$

Players choose a path from $s$ to $t$. There are five possible paths:
1. $P_1 = (e_1, e_3)$
2. $P_2 = (e_1, e_4)$
3. $P_3 = (e_2, e_3)$
4. $P_4 = (e_2, e_4)$
5. $P_5 = (e_5)$



In [ ]:
import pygambit as g

# Edge cost functions
def c1(x): return x + 33
def c2(x): return 3 * (x ** 2)
def c3(x): return 13 * x
def c4(x): return (x ** 2) + 44
def c5(x): return 47 * x

cost_funcs = {'e1': c1, 'e2': c2, 'e3': c3, 'e4': c4, 'e5': c5}

# The 5 possible paths from s to t
paths = {
    'P1': ['e1', 'e3'],
    'P2': ['e1', 'e4'],
    'P3': ['e2', 'e3'],
    'P4': ['e2', 'e4'],
    'P5': ['e5']
}

def compute_costs(path_p1, path_p2, w1, w2):
    """Compute the costs for player 1 and player 2 given their chosen paths and weights."""
    # Calculate load (x) on each edge
    load = {e: 0 for e in cost_funcs.keys()}
    for edge in paths[path_p1]: load[edge] += w1
    for edge in paths[path_p2]: load[edge] += w2
    
    # Calculate total cost for player 1
    cost1 = sum(cost_funcs[edge](load[edge]) for edge in paths[path_p1])
    
    # Calculate total cost for player 2
    cost2 = sum(cost_funcs[edge](load[edge]) for edge in paths[path_p2])
    
    return cost1, cost2

def build_congestion_game(w1, w2):
    """Generates the PyGambit bimatrix representation of the congestion game."""
    game = g.Game.new_table([len(paths), len(paths)])
    game.title = f"Congestion Game (w1={w1}, w2={w2})"
    game.players[0].label = f"Player 1 (w={w1})"
    game.players[1].label = f"Player 2 (w={w2})"
    
    path_names = list(paths.keys())
    for i, name in enumerate(path_names):
        game.players[0].strategies[i].label = name
        game.players[1].strategies[i].label = name

    # Fill the payoff matrix (payoffs are negative costs)
    for i, p1 in enumerate(path_names):
        for j, p2 in enumerate(path_names):
            c1, c2 = compute_costs(p1, p2, w1, w2)
            game[i, j][0] = -c1
            game[i, j][1] = -c2
            
    return game


## 2. Unweighted Case (Always has a Pure Equilibrium)

Let's consider the unweighted scenario where both Player 1 and Player 2 share a weight of 1 ($w_1 = w_2 = 1$). A well-known theoretical result establishes that every unweighted congestion game induces an exact potential game, meaning there is guaranteed to be at least one pure Nash equilibrium.



In [ ]:
# Unweighted Game: Player 1 has weight 1, Player 2 has weight 1
g_unweighted = build_congestion_game(w1=1, w2=1)

# Solve for pure strategy Nash equilibria
pure_eqs = g.nash.enumpure_solve(g_unweighted)

print(f"Found {len(pure_eqs)} pure Nash equilibrium/equilibria:")
for eq in pure_eqs:
    p1_strat = next(s.label for s in g_unweighted.players[0].strategies if eq[s] == 1)
    p2_strat = next(s.label for s in g_unweighted.players[1].strategies if eq[s] == 1)
    print(f"- Player 1 plays {p1_strat}, Player 2 plays {p2_strat}")


## 3. Weighted Case (No Pure Equilibrium)

Now, we introduce asymmetry. Suppose Player 1 has a weight of 1 ($w_1 = 1$), but Player 2 represents a larger group or higher-demand agent with a weight of 2 ($w_2 = 2$). 



In [ ]:
# Weighted Game: Player 1 has weight 1, Player 2 has weight 2
g_weighted = build_congestion_game(w1=1, w2=2)

pure_eqs_w = g.nash.enumpure_solve(g_weighted)
print(f"Found {len(pure_eqs_w)} pure Nash equilibria.")


As predicted by algorithmic game theory, this weighted formulation lacks a pure Nash equilibrium. However, Nash's existence theorem guarantees that a mixed equilibrium will exist.



In [ ]:
# Solve for mixed strategy Nash equilibria
mixed_eqs = g.nash.enummixed_solve(g_weighted)

print(f"Found {len(mixed_eqs)} mixed Nash equilibrium/equilibria:\n")
for eq in mixed_eqs:
    print("Player 1 mixed strategy:")
    for strat in g_weighted.players[0].strategies:
        prob = eq[strat]
        if prob > 0:
            print(f"  {strat.label}: {prob:.3f}")
            
    print("\nPlayer 2 mixed strategy:")
    for strat in g_weighted.players[1].strategies:
        prob = eq[strat]
        if prob > 0:
            print(f"  {strat.label}: {prob:.3f}")


## 4. Iterated Elimination of Strictly Dominated Strategies

One interesting property of finding equilibria is examining whether the game can be simplified. A rational player will never play a strictly dominated strategy. We can demonstrate **Iterated Elimination of Strictly Dominated Strategies (IESDS)** using PyGambit.



In [ ]:
import copy

# Create a copy so we don't mutate our original weighted game
g_weighted_reduced = copy.copy(g_weighted)

# Apply iterated elimination
g_weighted_reduced = g.game.simplify(g_weighted_reduced)

print("Original game size (strategies per player):")
print([len(p.strategies) for p in g_weighted.players])

print("\nReduced game size after elimination of strictly dominated strategies:")
print([len(p.strategies) for p in g_weighted_reduced.players])

# Let's inspect which strategies survived for each player
print("\nSurviving Strategies:")
for player in g_weighted_reduced.players:
    strategies = [s.label for s in player.strategies]
    print(f"- {player.label}: {', '.join(strategies)}")
